In [2]:
# Hidden TimeStamp
import time, datetime
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
print('Last Run: {}'.format(st))

Last Run: 2016-01-25 12:38:49


# The Package Architecture

The current package stems from a simple legacy script (circa 2014).  It has since been updated and currently abstracted with a focus to analyze more general laminated problems.  

The repository is extensibily designed for various geometry constructs given a specified, customized model.  This design architecture is diagrammed below.  

![API Diagram](./_images/diagram.png)

As shown, each diamond represents a module.  The diagram illustrates their relationships in passing objects between modules. The user-related areas are highlighted blue.  The package can be most actively extended in these blue areas.  The components of the lamana project can be distinguished as three types:

- Frontend: user-interacted, feature modules of particular interest that utilize laminate theory models
- Extension: directories/modules extending capabilities of the repository, e.g. `models` directory containing user defined laminate theories (`Classical_LT`, `Wilson_LT`).
- Backend: remaining Core modules, `input_`, `constructs_`, `theories_`, `output_`; workhorse facotries of LaminateModel objects.

## Package Module Summary

This section details some important modules critical to LamAna's operation.  The following table summarizes the core and feature modules in this package, what they intend to do and some important objects that result.  Objects that get passed between modules are italicized.

| Module | Classifier | Purpose     | Product |
|:------ |:---------- |:----------- |:-------:|
| `input_` | Backend   | Backend code for processing user inputs for all feature modules.  | *User Input object* i.e. `Geometry` |
| `distrubtions` | Feature | Analyze stress distributions for different geometries.  | *FeatureInput object*, `Case`, `Cases` | 
| `ratios` | Feature | Thickness ratio analyses for optimizing stress-geomtry design. | *FeatureInput object* |
| `predictions` | Feature | Failure predictions using experimental and laminate theory data. | *FeatureInput object* |
| `constructs` | Backend | Code for building `Laminate` objects.  | *LaminateModel object* |
| `theories` | Backend |  Code for selecting `Model` objects |  *Model* object |
| `<models>` | Extension | Directory of user-defined, custom LT models | *Model* objects |
| `output_` | Backend |  Code for several plotting objects, exporting and saving | Output object e.g. plots, xls, figures |


### Main Modules 

The modules listed above include Core (or Backend) modules and Feature modules.  The modules will be discussed in detail and include. 

The project is extensible in part by adding new Feature modules that integrate with the existing backend. Feature modules are the frontend of the repo because they are what users interact wih most.  However for simplicity, the Core modules and Features will coexist within a single directory.  The package file structure can be reviewed [on GitHub](https://github.com/par2/lamana/tree/develop/).

The Auxillary (or Utility) modules house support code that will not be discussed.

### Module Products

First we will mention some of the key inter-modular products will be mentioned briefly.  This objects can information that is exchanged between package modules.  This objects are illustrated as circles in the package [diagram].

#### FeatureInput

A FeatureInput is a Python dict that retains information from both a feature module and user-information processed by the `input_` module.  Here are the components:

| Key | Value | Description | 
|:---:|:-----:|:-----------:|
| `'Geometry'` | Geometry object | a single tuple of Geometry thicknesses |
|`'Parameters'`| load_params | loading parameters |
| `'Properties'` | mat_props | material properties, e.g. modulus, Poisson's ratio |
| `'Materials'` | materials index | ordered list of materials from DataFrame index |
| `'Model'` | model str | selected string of model name |
| `'Globals'` | None | a placeholder for future ubiquitous model variables |

`custom_matls` was depredated and replaced with `materials.`  The materials order is saved as a list and  interacts with the material cycler in `add_materials`.  It can be overwritten.  Parameters are associated with loading parameters.  Properties are associated with material properties.

```python
def make_FeatureInput(Geometry):
    '''Return a FeatureInput object.'''
    FI = {
        'Geometry': Geometry,                       # defined in Case      
        'Parameters': load_params,
        'Properties': mat_props,
        'Materials': materials,                     # set material order
        'Model': model,
        'Globals': None,                            # defined in theories/models
    }             
    return FI
```

#### LaminateModel

This is an `pandas` DataFrame object that combines data processed by the `constructs.Laminate` and `theories.<model>` classes.  The details of this object will be discussed further in the constructs section. 


---

## Core Module: `input_`

### `Geometry` class

This class is designed for parsing user-inputs and creating converted objects. For example, an input geometry string is formatted to a General Convention representing characteristic laminae types, i.e. outer-inner_i-middle.  An `Geometry object` is created of mixed Pythonic types - a namedtiple comprising floats and lists.

We distinguish the latter string and object types with the folllowing naming conventions:

- geometry string: raw string of the laminate geometry, e.g. `'400-200-800'`
- Geometry Object: `Geometry` class instance e.g. `<Geometry object (400-[200]-800)>`

Names referencing geometriy strings are lower-case: 

- `g`, `geo_inputs`, `geos` or `geos_full`, 
- `geos = ['400-[200]-800', '400-[100,100]-400S']`   

Names referencing **`Geometry` objects** are capatlizes: 

- `G`, `Geo_objects`, `Geos` or `Geos_full`, 
- `G = la.input_.Geometry(FeatureInput)`


```
    LamAna.input_.Geometry(geo_input) --> <Geometry object>
    
```


## Feature Module: `distributions`

### `Case` class

Blank

### `Cases` class

Blank

## Core Module: `constructs`

Principally, the `constructs` module builds a `LaminateModel` object, which currently a few `pandas` DataFrames representing a physical laminate and a few helpful attributes.  DataFrames were chosen as the backend object because they allow for powerful data manipulation analyses and later visualizations using common database/spreadsheet-like methods.  

Additionally, the `constructs` module computes laminate dimensional columns and compiles theoretical calculations handled by a sister `theories` module.  Conventiently, all of this data is contained in tabular form (the DataFrame).  The columns names are closely related to computational variables defined in the next sub-section.

### Variable Classifications

Here we distinguish two ubiquitous variable categories used internally.  These variables represented as columns in a full laminate DataFrame.  The categories for `LaminateModel` variables and columns are illustrated in the image below and described in greater detail:

![dataframe output](./_images/dataframe_output.png)

- **Laminate** (or `constructs`) variables: responsible for building the laminate stack and defining dimensions of the laminate.  Internally, these varibles will be distinguished with one trailing underscore.
    1. **ID**: variables related to layer and row identifications 
        1. `layer_`, `side_`, `matl_`, `type_`, `t_`
    2. **Dimensional**: variables of heights relative to cross-sectional planes
        1. `label_`, `h_`, `d_`, `intf_`, `k_`, `Z_`, `z_` 
- **Model** (or `theories`) variables: all remaining variables are relevant for LT calculations and defined from a given model. (Since these variables are model-specific, no particular naming format). However, for more detailed discussions, model variables can be further divided into sub-categories.  There common subsets are as follows:
    1. **User**:  global variables delibrately set by the user at startup
    2. **Inline**: variables used per lamina at a kth level (row)
    3. **Global**: variables applied to the laminate, accessible by ks

Image of the output for a DataFrame and there labeled categories of columns (IDs, dimensionals and models).  The first two categories are computed by `constructs.Laminate`.  The models columns are computed by `theories.models.<model_name>`, highlighted blue as it involves user interaction.  The layers are colored to assist in visualizing the data.

The finer granularity seen with model variables is not essential for typcial API use, but may be helpful when writing custom code that integrates with LamAna. 

### Further Details of Model Variables

Although model variables are pften particular to a chosen model, e.g Wilson_LT, the latter classifications are generic many modificied classical laminate theories.  Some model variables are provided at startup by the user (user_vars).  Some are calculated seen for each row of the data table (inline_vars).  Some variables are calculated by the designated laminate theory model, which provide constants for remaining calculations (global_vars).  As data displayed within columns, these Global variables would have the same number for every row.  Toredue redundancy, these constants are thus removed from the DataFrame but stored internally within a `dict`.   The details of this stored are coded within  a model module. Global values are of particular importance to `FeatureInput` objects (see later) and when exporting meta data as dashboards in spreadsheets. In contrast, Inline values alter directly with the dimensional values thoroughout the lamainate thickness. Names of common variables used in `distributions` are organized below:

*Model Variable Categories*

    Model_vars = {user_vars, inline_vars, global_vars}

*Subsets of Model Variables*

- user_vars   = [`mat_props`, `load_params`]
- global_vars = [`v_eq`, `D_11T`, `D_12T`, `M_r`, `M_t`, `D_11p`, `D_12n`, `K_r`, `K_t`]
- inline_vars = [`Q11`, `Q12`, `D11`, `D12`, `strain_r`, `strain_t`,
    `stress_r`, `stress_t`, `stress_f`]

Aside from User variables, all others are found as headers for columns 
in a DataFrame (or spreadsheet).

### The Laminate Architecture

This section will describe in greater detail how `LaminateModel`s are constructed.  When the user calls `case.apply()`, a series of objects are created.  We begin with a primitive `Stack`, which comprises skeletal components for building a Laminate DataFrame (also called an LFrame).  The phases for building a `LaminateModel` object are outlined below and structure the architexture of `constructs.Laminate` class.

- Phase 1: build a primitive laminate (Stack)
- Phase 2: calculate Laminate dimensional values (LFrame)
- Phase 3: calculate laminate theory Model values (LMFrame aka `LaminateModel`)

#### First: The `Stack` Class

The purpose of the `Stack` class is to build a skeletal, precusor of a primitive laminate object.  This class houses methods for parsing Geometry objects, ordering layers, adding materials labels for each layer and setting expected stress states for each tensile or compressive side.  `Stack` returns a namedtuple containing stack-related information (described below).

For a given `Geometry` object  instance (typically assigned to a "G" somewhere) the `constructs.Stack(G).StackTuple` method  creates a namedtuple of the stack information.  This object contains attributes to access the stack `order`, the number of plies (`nplies`), the technical `name` for the laminate, and a convenient `alias` if any.  The stack is a dict of the laminate layers ordered from bottom to top.  Now although dicts are unsorted, each layer is enumerated and stored as keys to retain the order, which preserves layer thickness and layer type (sometimes referred as "ltyype").  

```python
Example
=======
>>> import LamAna as la
>>> G = la.input_.Geometry(['400-200-800'])
>>> G
<Geometry object (400-[200]-800)>
>>> st = constructs.Stack(G).StackTuple    # converts G to a namedtuple
>>> st.order                               # access namedtuple attributes
{1: [400.0, 'outer'],
 2: [200.0, 'inner']
 3: [800.0, 'middle']
 4: [200.0, 'inner']
 5: [400.0, 'outer']}
>>> st.nplies
5
>>> st.name
'5-ply'
>>> st.alias
'standard'

```

More information can be found in the `Stack` class docstring.

#### Second: The `Laminate` Class

The `Laminate` class simply builds a `LaminateModel` - an object containing all dimensional information of a physical `Laminate` **and** all theoretical calculations using a laminate theory `Model`, e.g. stress/strain.

The `Laminate` class builds an object based on the skeletal layout of a stack parsed by and returned from the `Stack` class.  A `Geometry` object, material parameters and geometric parameters are all passed from the user in as a single  `FeatureInput` object - a dict of useful information that is passed between modules.  See *More on `FeatureInput`* for details.  Stack information is stored in an instance attribute called `Laminate.Snapshot` and then converted to a set of DataFrames.  

The IDs and dimensional data are determined and computed by `Stack` and `Laminate`.  Combined, this information builds an LFrame.  `Laminate` then calls the `theories` module which "handshakes" between the `Laminate` module and the custom module containing code of a user-specified, theoretical LT model.  (The "Model" is typically named auther after the author, sufixed by "`_LT`").   These computations update the Laminate DataFrame (`Laminate.LFrame`), creating a final LaminateModel (`Laminate.LMFrame`).  The workflow looks like the following.

Basic workflow:
```python
constructs :: class Stack --> class Laminate

theories :: class BaseModel
```

Laminate object + "Model" object --> LaminateModel object


Detailed workflow of `constructs-theories` interaction:
```python
class Stack --> StackTuple
 |
 |
class Laminate --> Snapshot, LFrame, LMFrame
 |
 | # Phase 1 : Instantiate; Determine Laminate ID Values
 | Laminate._build_snapshot(stack) --> Snapshot
 |     |
 |   Stack.add_materials(stack) 
 |   Stack.stack_to_df(stack)        # first creation of the Laminate df
 |   Laminate._set_stresses(stack)    
 |
 | Laminate._build_laminate(snapshot) --> LFrame
 | 
 | # Phase 2 : Calculate Laminate Dimensional Values
 | Laminate._update_columns._update_dimensions() --> LFrame (updated)
 |        label_, h_, d_, intf_, k_, z_, Z_
 |
 | # Phase 3 : Calculate Model Values
 | Laminate._update_columns._update_calculations() --> LMFrame
 |    theories.Model(Laminate)
 |    models.<selected model>
 |       _calc_stiffness()
 |       _calc_bending()
 |       _calc_moment()
 |       global_vars = [`v_eq`, `D_11T`, `D_12T`, ...]
 |       inline_vars = [`Q11`, `D11` `strain_r`, ...]
 |
LaminateModel : df
  
```

#### More on Material Stacking Order

The material order is defined by the user `mat_props` dict in `distributions` and automatically parsed in the `input_` module.  Extracting order from a dict is not trivial, so the default sorting is alphabetical order as handled by a pandas index.

As of 0.4.3d4, the user can partly override the default ordering by setting the `materials` property in the Case instance.  This allows simple control of the stacking order in the final laminate stack and `Laminate` objects. At the moment, the materials cycle through a list of materials; more customizations have not been implemented yet.

```python
>>> case.material
['HA', 'PSu']                                     # alphabetical order
>>> case.material = ['PSu', 'HA']                 # overriding order    
>>> case.material
['PSu', 'HA']
>>> case.apply(...)             
<materials DataFrame>                             # cycles the stacking order

```

#### More on `Laminate`

Using `Laminate._build_snapshot()`, the instance stack dict is converted to a DataFrame (`Snapshot`), giving a primitive view of the laminate geometry, idenfiers (IDs) and stacking order. This "snapshot" has the following ID columns of infornation, which are accessible to the user (see `distributions.Case.snapshot`):

    Variables addressed: `layer_, matl_, type_, t_`

From this snapshot, the DataFrame can is updated with new information.  For example, the sides on which to expected tensile and compressive stresses are located (`side_`) are assigned to a laminate through the `Laminate._set_stresses()` method.  This function accounts for DataFrames with even and odd rows.  For odd rows, 'None' is assigned to the neutral axis, implying "no stress".

    Variables addressed: `side_`

*Note, this stress assignment is a general designation, coarsely determined by which side of the netural axis a row is found.  The rigorous or finite stress state must be calculated through other analytical tools means such as Finite Element Analysis.*  

Likewise, the DataFrame is further updated with columns of dimensional data (Dimensional variables) and laminate theory data (Data variables).  The current LaminateModel object is made by calling `Laminate._update_columns._build_laminates()` which updates the snapshot columns to build two DataFrame objects:

See the similaritry between the laminate data columns and the its objects.

- `Snapshot`: primiate DataFrame of the Stack (see materials, layer info order).
- `LFrame`: updated `Snapshot` of IDs and dimensionals.
- `LMFrame`: updated LFrame with models computed columns.

![laminate objects](./_images/laminate_objects.png)

`LMFrame` is the paramount data structure of interest containing all IDs, Dimensional and Model variables and `p` number of rows pertaining to data points within a given lamina.  ~~It is up to the Laminate class to apply the appropriate Laminae types (Middle, Inner, Outer).~~ See *More on Laminae* to understand how laminae are determined.  The remaining variables will be discussed.

Dimensional variable columns are populated through the `Laminate._update_columns._update_dimensions()` method, which contains algorithms for calculating realative and absolute heights, thicknesses and midplane distances relative to the neutral axis.  These columns contain dimensional data that are determined independent from the laminate theory model. 

    Variables addresed: `label_, h_, d_, intf_, k_, Z_, z_`

These variables are defined in the Laminate class docstring.  See *More on label_* to understand the role of points, `p` and their relationship to DataFrame rows.  

Finally Data variable columns are populated using `Laminate._update_columns._update_calculations().`  These columnns contain data based on calculations from laminate theory for a selected model.  Here global_vars and inline_vars are calculated.

    Variables addressed:
    --------------------
    global_vars = [`v_eq, D_11T, D_12T, M_r, M_t, D_11p, D_12n, K_r, K_t`] --> FeatureInput['Global'] (dict entry)
    
    inline_vars = [`Q11, Q12, D11, D12, strain_r, strain_t, stress_r, stress_t, stress_f`] --> LaminateModel object (DataFrame)
    
##### More on Laminae

Definitions:

- Lamina - *singular* a single laminate layer
- Laminae - *plural* multiple laminate layers
- Laminate - a structure comprising stacked laminae

A lamina is essentially a layer from a `Stack` with `p` number of DataFrame rows.  These rows represent points within a lamina that reflect inline calculations detemined in laminate theory, e.g. stress or strain.  Laminae have either **unbound interfaces** (exposed surfaces), or **bound interfaces**, proximal to an adjacent lamina.  For example, outer layers have one bound and one unbound interface, while inner layers have two bound interfaces.

~~OuterLayer and InnerLayer are subclasses of Laminate.  They are unique because their stress states are must be determined by their position in the Laminate stack order.  This stress `side` is passed in through `Laminate.buid.rebuild(*args)`. MiddleLayer is also subclassed from Laminae, but instances assign stress sides differently.~~  "None" is assigned to rows pertaining to neutral axes where the stress is neither compressive or tensile.  Moreover, it is not possible to differentiate a single row into tensile ("Tens.") and compressive ("Comp.") sides.  For such indeterminate rows, the value "INDET" is assigned; these apply to odd plies with p = 1.  For p > 1, MiddleLayer has both tensile and compressive stress states.  ~~For this reason, Laminate.set_stresses() is called separately of MiddleLayer.  This issues is discussed further in *More on `label_`*~~

#### More on `FeatureInput`

A Feature module defines a `FeatureInput` object. For `distributions`, it is defined in `Case`. `FeatureInput`s contain information that is passed between objects.  For instance, this object transfers user input data in `distributions` (converted in `input_`) to the `constructs` module to build the laminate stack and populate ID and dimensional columns. A FeatureInput from `distributions` looks like the following (as of 0.4.4b).
```python
FeatureInput = {'Geometry': <Geometry object>,
                'Loading': <load_params dict>,
                'Materials': <mat_props dict>,
                'Custom': <undefined>,
                'Model': <string>,
                'Globals': <dict>
                }
```                    
After calculating model data, the Globals key is updated containing all necessary `globabl_vars`.  These variables are constant and are necessary for further calculations of `inline_vars`.

```python
FeatureInput['Globals'] = [v_eq, D_11T, D_12T, M_r, M_t, D_11p, D_12n, K_r, K_t]
```

#### More on `label_`

See LPEP 001.02 for API unit standards.

We transverse the absolute height of the laminate at different cross-sectional planes.  The values of inline stress points are calculated along different planes throughout the laminate thickness. What happens at interfaces where two materials meet with different stresses?  How are these two stress points differentiated in a DataFrame or in a plot?  For plotting purposes, we need to define diferent types of points.  Here we define some rulse and four types of points found within a (i.e. DataFrame rows):

1. interfacial - point on unbound outer surfaces and bound internal surfaces.
2. internal - point with the lamina thickness between interfaces 
3. discontinuity - point on bounded interfaces pertaining to an adjacent lamina
4. neutralaxis - the middle, symmetric axial plane

How these points are distributed depends on their locations within each lamina and whether they are located on the tensile or compressive `side_`.  ~~InnerLayer and OuterLayer acquire stress information from the stack (see `Stack.stack_to_df`) since the points are mirrored across the neutral axis.~~  The neutral axis exists in physical laminates, but they are only represented as a row in DataFrames of odd ply, odd p laminates.  ~~Since the MiddleLayer must always intersect the position of the neutral axis, its construction is more predictable.  Thus MiddleLayer uses `Laminate.set_stress()` independently to distribute points.~~  The image below differentiates between grups of point found in middle layers and any other layer with respect to `k_`.

![points](./_images/points.png)

Notice various layers have different point types.

- Middle layers have two interfacial points, no discontinuities and a neutral axis.
- All other layers have one interfacial point with a discontinuity if p > 2.
- All layers may have internal points.

*Note: only the interfacial points can be theoreticlly verified, representing the maximum prinicipal stresses and strains.  The internal and discontinuity points are merely used by matplotlib to connect the points that **assume a linear stress distribution.**  

~~*Note: the calcuations for `z_` assumes a linear distribution of points.  It requires `h_` discontinuties to adopt the adjacent layer's value.  The accuracy of these assumptions shoulbe be noted and reevaluated for more robust calculations in the future.  A post-processing loop was used to amend these particular points.*~~

*Note: the midplane z height (`z_`) for discontinuities assumes a non-zero, lower limit value equal to the Z_ height of the bounding layer.  This value should be verified.*

## Core Module: `theories`

### `LaminateModel` Handling

An illustration of LaminateModel handling is shown below.  The `Laminate` DataFrame (LFrame) is passed from `constructs` to `theories`.  If successful the `LaminateModel` is returned to `constructs`; otherwise the `Laminate` is returned unchanged (LFrame).  

![theories flowchart](./_images/diagram_theories.png)
*NOTE: The term repr for <`LaminateModel object`> remains constant refering to a post-theories operation, whether LMFrame is updated with Model columns or not.*

When `Laminate._update_columns._update_calculations()` is called, an instance of `self` (x) is passed to `theories.handshake()` (black arrow).  This function handles all updates to the primitive `Laminate` DataFrame (LFrame) which comprise IDs and Dimensional columns only.  `self` gives the model's author full access to Laminate attributes.  From here, `theories.handshakes()` searches within the models directory for a model (grey, dashed arrow) specified by the user at the time of instantiation, i.e. `Case.apply(*args, model=<model_name>).` 

A model is simply a module containing code that handles Laminate Theory calculations.  The purpose of the model is to update the primitive DataFame with LT calculations.  `handshake()` distinguishes whether the author implemented a class-style or function-style model.  

The **most important hook method/function is `_use_model_()`**, which must be present somewhere in the model module and must return a tuple containing:
    - the updated Laminate DataFrame with model data columns (a.k.a. `LaminateModel`)
    - the `FeatureInput` with updated 'Globals' key - a dict of calculated constants, used in exported reports (see output_ section). 

Finally, the `Laminate.LMFrame` attribute is updated with the new `LaminateModel` DataFrame (green arrow).  However, if exceptions are raised, `Laminate._update_calculations()` handles reverting the LMFrame to a copy of LFrame, printing a warning and minor traceback informing the author to refactor the code.  This is commom for Laminates with p=1, which detects an INDET in middle layers and must revert to LFrame.   See Exceptions for more details.   

### Custom Models

A powerful, extensible option of the `LamAna` package is user implementation of their own laminate theory models.  A library of these custom models (as well as the defaults) are kept in the `models` directory (sub-package).  This is possbile since the `theories` module handshakes between the `constructs` module and the selected model from the `models` sub-package.  All models related exceptions and global model code is housed in `theories` and merges the model calculations to generate data variables in the `LaminateModel` object.  

## Core Module: `output_`

A summary of `output` objects

| Object | Purpose |
|:------ |:-------- |
| `SinglePlot` | Stress distribution for a single geometry |
| `MultiPlot` | Stress distributions for a multiple geometries |
| `HalfPlot` | Partial plot of either compression or tension side |
| `QuarterPlot` | Partial halfplot excluding side without data |
| `PanelPlot` | A series of subplots side-by-side |
| `RatioPlot` | Ratio thickness plot; prinicipal stress vs. ratio |
| `PredictPlot` | Plot of experimental failure load or stress vs. middle layer princ. stress |
